In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy  import sparse
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
from pprint import pprint

In [2]:
rating = pd.read_csv("../input/recommednation-system/rating.csv")
places = pd.read_csv("../input/recommednation-system/placeid.csv")


In [3]:
places.head()

In [4]:
rating.head()

In [5]:
print('Number of places : ',  len(places))
print('Number of Ratign : ',len(rating))
print('Number of Users :' ,  len(rating['user id'].unique()))

In [6]:
rating.dropna(inplace= True )
rating = rating.iloc[:,3:]
rating.head()

In [7]:
rating.describe()

In [8]:
#rating.drop(["user id","place id"],inplace =True ,axis=1)

In [9]:
rating.head()

In [10]:
fig, ax = plt.subplots()
ax.bar("zero",rating[rating['rate'] == 0 ].count(),label="zero",color="red")
ax.bar("one",rating[rating['rate'] == 1 ].count(),label="one",color="blue")
ax.bar("two",rating[rating['rate'] == 2 ].count(),label="tww",color="green")
ax.bar("three",rating[rating['rate'] == 3 ].count(),label="three",color="yellow")
ax.bar("four",rating[rating['rate'] == 4 ].count(),label="four",color="black")
ax.bar("five",rating[rating['rate'] == 5 ].count(),label="five",color="orange")
ax.set_ylabel('Number of ratings')
ax.set_xlabel('Rating range')

In [11]:
user_to_place = rating.pivot(
    index='user id',
     columns='place id',
      values='rate')

user_to_place.head()

In [12]:
user_to_place.to_csv("user_to_place.csv")

In [13]:
user_to_place =  pd.read_csv("./user_to_place.csv")

In [14]:
(user_to_place.columns)

In [15]:
user_to_place_sparse = csr_matrix(user_to_place.values)
user_to_place_sparse

In [16]:
user_to_place_sparse.get_shape()

In [17]:
sparse.save_npz("sparcematrix.npz", user_to_place_sparse)
user_to_place_sparse = sparse.load_npz("./sparcematrix.npz")

In [18]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_to_place_sparse)

In [19]:
user_dict = {user : index for index, user in enumerate(list(rating["user_id"])) }
c = 0
for i,j in user_dict.items():
    user_dict[i] = c 
    c+=1
user_dict

In [20]:
with open('user_dict', 'wb') as fp:
    pickle.dump(user_dict, fp)

In [21]:
with open ('./user_dict', 'rb') as fp:
    place_dict = pickle.load(fp)

user_dict  

In [22]:
place_dict = {place : index for index, place in enumerate(list(places["Places"]))}
#print(place_dict)

In [23]:
with open('place_dict', 'wb') as fp:
    pickle.dump(place_dict, fp)

In [24]:
with open ('./place_dict', 'rb') as fp:
    place_dict = pickle.load(fp)

place_dict  

In [25]:
place_dictionary = {}
for i in  range(len(places)):
    place_dictionary[places.iloc[i,2]] = places.iloc[i,-1]
#print(place_dictionary)

In [26]:
with open('place_dictionary', 'wb') as fp:
    pickle.dump(place_dictionary, fp)

In [27]:
with open ('./place_dictionary', 'rb') as fp:
    place_dictionary = pickle.load(fp)

place_dictionary  

In [28]:
place_list = list(places.Places)

In [29]:
#print(place_list)
with open('place_list', 'wb') as fp:
    pickle.dump(place_list, fp)
    

In [30]:
with open ('./place_list', 'rb') as fp:
    place_list = pickle.load(fp)

place_list  

In [31]:
def get_similar_places(place, n = 10):
    recommender = []
    ID = []
    place =  [k for k,v in place_dictionary.items() if v == place][0]
    index = place_dict[place]
    knn_input = np.asarray([user_to_place.values[index]])
    n = min(len(place_list)-1,n)
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
  
    print("Top",n,"places which are very much similar to the place-",place, "are: ")
    print(" ")
    for i in range(1,len(distances[0])):
        recommender.append(place_list[indices[0][i]])
        ID.append(place_dictionary[recommender[-1]])
    return {"Recommended Places Names" :recommender , "Recommender Places ID": ID}

In [32]:
#place_name = 'The Egyptian Museum In Cairo'
place_name = "6249c7df30170c0174c4419a"
print(get_similar_places(place_name,9))

In [33]:
def get_similar_users(user, n = 2):
    result = []
    users = []
    user =  user_dict[user]
    knn_input = np.asarray([user_to_place.values[user-1]]) 
    distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
  
    #print("Top",n,"users who are very much similar to the User-",user, "are: ")
    #print(" ")
    for i in range(1,len(distances[0])):
        users.append(indices[0][i]+1)
        #result.append({"user":indices[0][i]+1,"space":distances[0][i]})
    for i in range(n):
        for place in (list(rating[(rating['user id'] == users[i]) & (rating["rate"] > 3)]['place name'])[:5]):
             if place not in result :
                result.append({"place_name": place , "place_id" : place_dictionary[place] })
        
    return {"top places for similar users": result} 

In [35]:
user_id = "62cb03b08ca3fa1264296dd7"
#pprint((list(rating[(rating['user id'] == user_id) & (rating["rate"] > 3)]['place name'])[:5]))
results  = (get_similar_users(user_id,2))
print(results)